In [1]:
import numpy as np

from networks import FiniteFieldPiNetNetworkLinear, FiniteFieldPiNetNetworkLeNet, FiniteFieldPiNetNetworkLeNetCIFAR10, FiniteFieldPiNetNetworkDebug
from criterions import FiniteFieldMSELoss
from datasets import load_all_data_mnist, load_all_data_cifar10
from utils import create_batch_data, to_real_domain

In [2]:
BATCH_SIZE = 256
EPOCH = 1
LR = 7
PRINT = 10
MODE = 'debug'
PRIME = 2**26 - 5
QUANTIZATION_WEIGHT = 8
QUANTIZATION_INPUT = 8
QUANTIZATION_BATCH_SIZE = 8

In [3]:
if MODE == 1:
    model = FiniteFieldPiNetNetworkLinear(QUANTIZATION_WEIGHT, PRIME, QUANTIZATION_INPUT)
    flatten = True
elif MODE == 2:
    model = FiniteFieldPiNetNetworkLeNet(QUANTIZATION_WEIGHT, PRIME, QUANTIZATION_INPUT)
    flatten = False
elif MODE == 3:
    model = FiniteFieldPiNetNetworkLeNetCIFAR10(QUANTIZATION_WEIGHT, PRIME, QUANTIZATION_INPUT)
    flatten = False
elif MODE == 'debug':
    model = FiniteFieldPiNetNetworkDebug(QUANTIZATION_WEIGHT, PRIME, QUANTIZATION_INPUT)
    flatten = False
else:
    model = None
    flatten = True
criterion = FiniteFieldMSELoss(PRIME, QUANTIZATION_WEIGHT, QUANTIZATION_BATCH_SIZE)

In [4]:
# data fetching
load_path = '../../data'
train_data, train_label, test_data, test_label = load_all_data_mnist(load_path, QUANTIZATION_INPUT, QUANTIZATION_WEIGHT, PRIME, flatten=flatten)
train_data, train_label, test_data, test_label = create_batch_data(train_data, train_label, test_data, test_label, BATCH_SIZE)

In [ ]:
for epoch in range(EPOCH):
    tot_loss = 0
    for train_idx, (train_data_batch, train_label_batch) in enumerate(zip(train_data, train_label)):
        # train
        preds = model.forward(train_data_batch)

        loss = criterion.forward(preds, train_label_batch)
        tot_loss += loss
        propagated_error = criterion.error_derivative()

        model.backprop(propagated_error)
        model.optimize(LR)
        print('idx: {}, loss: {}'.format(train_idx + 1, loss))
        if (train_idx + 1) % PRINT == 0:
            print('epoch: {}, idx: {}, loss: {}'.format(epoch + 1, train_idx + 1, tot_loss / PRINT))
            tot_loss = 0

        if (train_idx + 1) % PRINT == 0:
            tot_acc = 0
            tot_sample = 0
            for train_acc_idx, (test_data_batch, test_label_batch) in enumerate(zip(test_data, test_label)):
                # train accuracy
                preds = model.forward(test_data_batch)
                real_preds = to_real_domain(preds, QUANTIZATION_WEIGHT, PRIME)
                pred_args = np.argmax(real_preds, axis=1)

                tot_acc += np.count_nonzero(pred_args == test_label_batch)
                tot_sample += test_data_batch.shape[0]

            accuracy = tot_acc / tot_sample
            if train_idx != 0:
                tot_loss = tot_loss / PRINT
            print('epoch: {}, idx: {}, accuracy: {}, loss: {}'.format(epoch + 1, train_idx + 1, accuracy, tot_loss))
            tot_loss = 0

idx: 1, loss: 2.8752511739730835
idx: 2, loss: 1.820149004459381
idx: 3, loss: 3.1006760597229
idx: 4, loss: 2.352128386497498
idx: 5, loss: 1.8502511978149414
idx: 6, loss: 0.9645739197731018
idx: 7, loss: 0.8610182404518127
idx: 8, loss: 0.9194228649139405
idx: 9, loss: 0.805298924446106
idx: 10, loss: 0.7365047931671143
epoch: 1, idx: 10, loss: 1.6285274565219878
epoch: 1, idx: 10, accuracy: 0.5838, loss: 0.0
idx: 11, loss: 0.7790211439132689
idx: 12, loss: 0.8088868856430053
idx: 13, loss: 0.7293991446495056
idx: 14, loss: 0.811723232269287
idx: 15, loss: 0.7457399964332582
idx: 16, loss: 0.6620576977729797
idx: 17, loss: 0.6312770843505859
idx: 18, loss: 0.6187778115272522
idx: 19, loss: 0.6908708214759827
idx: 20, loss: 0.615646719932556
epoch: 1, idx: 20, loss: 0.7093400537967682
epoch: 1, idx: 20, accuracy: 0.7135, loss: 0.0
idx: 21, loss: 0.598652720451355
idx: 22, loss: 0.5760615468025208
idx: 23, loss: 0.5898118615150453
idx: 24, loss: 0.6099125146865845
idx: 25, loss: 0.628